In [ ]:
# Montar Drive e imports
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from pathlib import Path
import zipfile
import os

In [ ]:
# Caminho base do projeto
BASE_DIR = Path('/content/drive/MyDrive/UTP_ATM_V3')

# ZIP do CVAT
ZIP_PATH = BASE_DIR / 'YOLO/CONDILOS_ROI_A_N.zip'

# Pasta extrair o ZIP
RAW_DIR = BASE_DIR / 'YOLO/cvat_yolo_condyle_raw'
RAW_DIR.mkdir(parents=True, exist_ok=True)

print("📦 ZIP_PATH:", ZIP_PATH)
print("📂 RAW_DIR:", RAW_DIR)

# Extrair ZIP
with zipfile.ZipFile(ZIP_PATH, 'r') as z:
    z.extractall(RAW_DIR)

print("\n✨ ZIP extraído com sucesso!")
print("📁 Conteúdo de RAW_DIR:", os.listdir(RAW_DIR))


Mounted at /content/drive
📦 ZIP_PATH: /content/drive/MyDrive/UTP_ATM_V3/YOLO/CONDILOS_ROI_A_N.zip
📂 RAW_DIR: /content/drive/MyDrive/UTP_ATM_V3/YOLO/cvat_yolo_condyle_raw

✨ ZIP extraído com sucesso!
📁 Conteúdo de RAW_DIR: ['train.txt', 'obj.names', 'obj.data', 'obj_train_data']


In [ ]:
import os

# imagens + labels do CVAT
RAW_LABEL_DIR = RAW_DIR / 'obj_train_data'

print("📁 Existe obj_train_data?", RAW_LABEL_DIR.exists())
if RAW_LABEL_DIR.exists():
    print("📄 Conteúdo (amostra):", os.listdir(RAW_LABEL_DIR)[:10])
else:
    print("⚠️ obj_train_data ainda NÃO existe — confira a estrutura extraída do ZIP!")

# Pastas com as imagens ROIs originais
DIR_A_ROI = BASE_DIR / 'A_ROI'
DIR_N_ROI = BASE_DIR / 'N_ROI'

print("\n📂 BASE_DIR:", BASE_DIR)
print("📂 DIR_A_ROI:", DIR_A_ROI)
print("📂 DIR_N_ROI:", DIR_N_ROI)
print("📂 RAW_LABEL_DIR:", RAW_LABEL_DIR)

if RAW_LABEL_DIR.exists():
    print("\n📁 Conteúdo de RAW_LABEL_DIR (amostra):")
    print(os.listdir(RAW_LABEL_DIR)[:10])

📁 Existe obj_train_data? True
📄 Conteúdo (amostra): ['img1A_padronizado_ROI.txt', 'img40N_padronizado_ROI.txt', 'img4N_padronizado_ROI.txt', 'img100A_padronizado_ROI.txt', 'img98A_padronizado_ROI.txt', 'img22N_padronizado_ROI.txt', 'img10A_padronizado_ROI.txt', 'img89A_padronizado_ROI.txt', 'img13N_padronizado_ROI.txt', 'img31N_padronizado_ROI.txt']

📂 BASE_DIR: /content/drive/MyDrive/UTP_ATM_V3
📂 DIR_A_ROI: /content/drive/MyDrive/UTP_ATM_V3/A_ROI
📂 DIR_N_ROI: /content/drive/MyDrive/UTP_ATM_V3/N_ROI
📂 RAW_LABEL_DIR: /content/drive/MyDrive/UTP_ATM_V3/YOLO/cvat_yolo_condyle_raw/obj_train_data

📁 Conteúdo de RAW_LABEL_DIR (amostra):
['img1A_padronizado_ROI.txt', 'img40N_padronizado_ROI.txt', 'img4N_padronizado_ROI.txt', 'img100A_padronizado_ROI.txt', 'img98A_padronizado_ROI.txt', 'img22N_padronizado_ROI.txt', 'img10A_padronizado_ROI.txt', 'img89A_padronizado_ROI.txt', 'img13N_padronizado_ROI.txt', 'img31N_padronizado_ROI.txt']


In [ ]:
from pathlib import Path

VALID_EXTS = ['.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff']

label_files = sorted([p for p in RAW_LABEL_DIR.glob('*.txt')])
print(f"🔎 Total de labels (.txt) encontrados: {len(label_files)}")

pairs = []        # (img_path, label_path)
missing_imgs = [] # labels sem imagem correspondente

for lbl in label_files:
    stem = lbl.stem  # ex: img1A_padronizado_ROI
    img_path = None

    # Tenta achar em A_ROI
    for ext in VALID_EXTS:
        candidate = DIR_A_ROI / f"{stem}{ext}"
        if candidate.exists():
            img_path = candidate
            break

    # Se não achou em A_ROI, tenta em N_ROI
    if img_path is None:
        for ext in VALID_EXTS:
            candidate = DIR_N_ROI / f"{stem}{ext}"
            if candidate.exists():
                img_path = candidate
                break

    if img_path is None:
        missing_imgs.append(stem)
    else:
        pairs.append((img_path, lbl))

print(f"✅ Pares imagem+label encontrados: {len(pairs)}")
print(f"⚠️ Labels sem imagem correspondente: {len(missing_imgs)}")

if missing_imgs:
    print("\nExemplos de labels sem imagem:")
    for s in missing_imgs[:10]:
        print(" -", s)

print("\nExemplos de pares (img, label):")
for img_path, lbl_path in pairs[:5]:
    print("🖼", img_path.name, "<->", lbl_path.name)

🔎 Total de labels (.txt) encontrados: 200
✅ Pares imagem+label encontrados: 200
⚠️ Labels sem imagem correspondente: 0

Exemplos de pares (img, label):
🖼 img100A_padronizado_ROI.png <-> img100A_padronizado_ROI.txt
🖼 img100N_padronizado_ROI.png <-> img100N_padronizado_ROI.txt
🖼 img10A_padronizado_ROI.png <-> img10A_padronizado_ROI.txt
🖼 img10N_padronizado_ROI.png <-> img10N_padronizado_ROI.txt
🖼 img11A_padronizado_ROI.png <-> img11A_padronizado_ROI.txt


In [ ]:
import shutil
import random

DATA_DIR = BASE_DIR / 'YOLO/yolo_condyle_roi'

IMG_TRAIN_DIR = DATA_DIR / 'images' / 'train'
IMG_VAL_DIR   = DATA_DIR / 'images' / 'val'
LBL_TRAIN_DIR = DATA_DIR / 'labels' / 'train'
LBL_VAL_DIR   = DATA_DIR / 'labels' / 'val'

# (Opcional) se quiser limpar tudo que tinha antes, descomentar:
# if DATA_DIR.exists():
#     shutil.rmtree(DATA_DIR)

# Criar as pastas
for d in [IMG_TRAIN_DIR, IMG_VAL_DIR, LBL_TRAIN_DIR, LBL_VAL_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("📁 Estrutura criada em:", DATA_DIR)

# Embaralhar os pares e separar treino/val
random.shuffle(pairs)

val_ratio = 0.2  # 20% para validação
val_count = int(len(pairs) * val_ratio)

val_pairs = pairs[:val_count]
train_pairs = pairs[val_count:]

print(f"📊 Treino: {len(train_pairs)} imagens")
print(f"📊 Validação: {len(val_pairs)} imagens")


📁 Estrutura criada em: /content/drive/MyDrive/UTP_ATM_V3/YOLO/yolo_condyle_roi
📊 Treino: 160 imagens
📊 Validação: 40 imagens


In [ ]:
def copy_pair(img_path, lbl_path, img_dst_dir, lbl_dst_dir):
    shutil.copy2(img_path, img_dst_dir / img_path.name)
    shutil.copy2(lbl_path, lbl_dst_dir / lbl_path.name)

# Copiar treino
for img_path, lbl_path in train_pairs:
    copy_pair(img_path, lbl_path, IMG_TRAIN_DIR, LBL_TRAIN_DIR)

# Copiar validação
for img_path, lbl_path in val_pairs:
    copy_pair(img_path, lbl_path, IMG_VAL_DIR, LBL_VAL_DIR)

print("✅ Cópia de imagens e labels concluída.")


✅ Cópia de imagens e labels concluída.


In [ ]:
data_yaml_path = DATA_DIR / 'data.yaml' #data.yaml contém: imagens de treino e validação, informação de classes, índices

yaml_text = f"""
path: {DATA_DIR}

train: images/train
val: images/val

names:
  0: Normal
  1: Achatado
"""

with open(data_yaml_path, 'w', encoding='utf-8') as f:
    f.write(yaml_text.strip() + "\n")

print("✅ data.yaml criado em:", data_yaml_path)

print("📂 DATA_DIR:", DATA_DIR)
print("📁 images/train (qtd):", len(os.listdir(IMG_TRAIN_DIR)))
print("📁 images/val   (qtd):", len(os.listdir(IMG_VAL_DIR)))
print("📁 labels/train (qtd):", len(os.listdir(LBL_TRAIN_DIR)))
print("📁 labels/val   (qtd):", len(os.listdir(LBL_VAL_DIR)))

print("\n📄 Conteúdo de data.yaml:\n")
with open(data_yaml_path, 'r') as f:
    print(f.read())


✅ data.yaml criado em: /content/drive/MyDrive/UTP_ATM_V3/YOLO/yolo_condyle_roi/data.yaml
📂 DATA_DIR: /content/drive/MyDrive/UTP_ATM_V3/YOLO/yolo_condyle_roi
📁 images/train (qtd): 160
📁 images/val   (qtd): 40
📁 labels/train (qtd): 160
📁 labels/val   (qtd): 40

📄 Conteúdo de data.yaml:

path: /content/drive/MyDrive/UTP_ATM_V3/YOLO/yolo_condyle_roi

train: images/train
val: images/val

names:
  0: Normal
  1: Achatado

